In [1]:
import os
import numpy as np
from preprocessing import *
from equalization import *
%load_ext autoreload
%autoreload 2

In [10]:
class Signal:
	def __init__(self, path, window_size, hop_length):
		self.path = path
		self.sr = librosa.get_samplerate(self.path)
		self.window_size = window_size
		self.hop_length = hop_length
		self.signal, _ = librosa.load(self.path, sr=self.sr)
		self.fft = np.abs(librosa.core.stft(self.signal, n_fft=self.window_size, hop_length=self.hop_length))
		self.freq_bins = self.fft.shape[0]
		self.fft_db = librosa.amplitude_to_db(self.fft)

	def set_chunk(self, seconds):
		fft_length = self.fft_db.shape[1]
		num_freqs = self.fft_db.shape[0]
		chunk_size = int(np.ceil((1 / (self.window_size / self.sr)) * seconds))
		total_chunks = int(np.ceil(fft_length / chunk_size))
		avg_mat = np.zeros((num_freqs, total_chunks))
		avg_vec = np.zeros((1, chunk_size))
		for i in range(num_freqs):
			for j in range(total_chunks):
				if j > total_chunks - 1:
					avg_vec = self.fft_db[i][chunk_size * j:]
					mu = np.mean(avg_vec)
					avg_mat[i][j] = mu
				avg_vec = self.fft_db[i][chunk_size * j: chunk_size * (j+1)]
				mu = np.mean(avg_vec)
				avg_mat[i][j] = mu
		self.chunk_fft = avg_mat

	def set_rank_2d(self): 
		a = np.zeros(self.chunk_fft.shape)
		for row in range(self.chunk_fft.shape[1]):
			a[:, row] = np.abs(rankdata(self.chunk_fft[:, row], method='min') - (self.chunk_fft.shape[0])) + 1
		self.rank = a

	def set_sparsity(self):
		sparse_vec = np.zeros((1, self.rank.shape[1]))
		min_val = self.freq_bins
		for i in range(self.rank.shape[1]):
			mu = np.mean(self.rank.T[i])
			if mu == min_val:
				sparse_vec[0, i] = 0
			else:
				sparse_vec[0, i] = 1
		self.sparse_vec = sparse_vec

	def overlap(self, sv1):
		overlap_vec = self.sparse_vec * sv1
		num_overlaps = np.sum(overlap_vec)
		overlap_ratio = num_overlaps / overlap_vec.shape[1]
		return overlap_vec, num_overlaps, overlap_ratio

	def sparse_overlap_avg(self, overlap_vec, num_overlaps):
		soa_vec = np.zeros((self.freq_bins, 1))
		for i in range(self.freq_bins):
			soa_vec[i] = np.sum((self.chunk_fft * self.sparse_vec) * overlap_vec) / num_overlaps
		return soa_vec

	def rank_soa_vec(self, soa_vec): return np.abs(rankdata(soa_vec, method='min') - (soa_vec.shape[0])) + 1

	def masker_rank_vec(self, r_soa_vec): np.expand_dims(np.where(r_soa_vec > 10, 1, 0), axis=1)

	def maskee_rank_vec(self, r_soa_vec): np.expand_dims(np.where(r_soa_vec <= 10, 1, 0), axis=1)

In [13]:
Signal(path, window_size=1024, hop_length=1024)

In [2]:
path = '../Downloads/mozzy_type'
files = file_scraper(path)
full_paths = [path + '/' + file for file in files]

In [23]:
devon = mask_chunks(paths=full_paths, window_size=1024, hop_length=1024, seconds=5, rank_threshold=10, max_n=3, min_overlap_ratio=0.10, max_eq=5)

In [24]:
devon

[{'../Downloads/mozzy_type/Mozzy Type - Chromosome_Gm_01.wav': [[1719.298245614035,
    4.43753440055673],
   [1375.438596491228, 4.786612209827602],
   [1289.4736842105262, 5.0]]},
 {'../Downloads/mozzy_type/Mozzy Type - Impact St 1.wav': [[1719.298245614035,
    4.43753440055673],
   [1375.438596491228, 4.786612209827602],
   [1289.4736842105262, 5.0]]},
 {'../Downloads/mozzy_type/Mozzy Type - Clean Reverse Cym.wav': [[1719.298245614035,
    4.43753440055673],
   [1375.438596491228, 4.786612209827602],
   [1289.4736842105262, 5.0]]},
 {'../Downloads/mozzy_type/Mozzy Type - Metro SFX 23.wav': [[1719.298245614035,
    4.43753440055673],
   [1375.438596491228, 4.786612209827602],
   [1289.4736842105262, 5.0]]},
 {'../Downloads/mozzy_type/Mozzy Type - AhVedonde (efx2).wav': [[1719.298245614035,
    4.43753440055673],
   [1375.438596491228, 4.786612209827602],
   [1289.4736842105262, 5.0]]},
 {'../Downloads/mozzy_type/Mozzy Type - K.E on the track  808 BD (194).wav': []},
 {'../Downloads/

In [26]:
save_signals(devon, files, 2, "../Downloads/devon1", "EQ - ")

Pyo message: Offline Server rendering file ../Downloads/devon1/EQ - Mozzy Type - Chromosome_Gm_01.wav dur=149.333313
Pyo message: Offline Server rendering finished.
Pyo message: Offline Server rendering file ../Downloads/devon1/EQ - Mozzy Type - Impact St 1.wav dur=149.333313
Pyo message: Offline Server rendering finished.
Pyo message: Offline Server rendering file ../Downloads/devon1/EQ - Mozzy Type - Clean Reverse Cym.wav dur=149.333313
Pyo message: Offline Server rendering finished.
Pyo message: Offline Server rendering file ../Downloads/devon1/EQ - Mozzy Type - Metro SFX 23.wav dur=149.333313
Pyo message: Offline Server rendering finished.
Pyo message: Offline Server rendering file ../Downloads/devon1/EQ - Mozzy Type - AhVedonde (efx2).wav dur=149.333313
Pyo message: Offline Server rendering finished.
Pyo message: Offline Server rendering file ../Downloads/devon1/EQ - Mozzy Type - K.E on the track  808 BD (194).wav dur=149.333313
Pyo message: Offline Server rendering finished.
Pyo 

In [11]:
path = '../Downloads/mozzy_type/Mozzy Type - Presence 5.wav'
path1 = '../Downloads/mozzy_type/Mozzy Type - Bad Guy Kick.wav'


In [43]:
class Signal:
	def __init__(self, path, window_size, hop_length):
		self.path = path
		self.sr = librosa.get_samplerate(self.path)
		self.window_size = window_size
		self.hop_length = hop_length
		self.signal, _ = librosa.load(self.path, sr=self.sr)
		self.fft = np.abs(librosa.core.stft(self.signal, n_fft=self.window_size, hop_length=self.hop_length))
		self.freq_bins = self.fft.shape[0]
		self.fft_db = librosa.amplitude_to_db(self.fft)

	def set_chunk(self, seconds):
		fft_length = self.fft_db.shape[1]
		num_freqs = self.fft_db.shape[0]
		chunk_size = int(np.ceil((1 / (self.window_size / self.sr)) * seconds))
		total_chunks = int(np.ceil(fft_length / chunk_size))
		avg_mat = np.zeros((num_freqs, total_chunks))
		avg_vec = np.zeros((1, chunk_size))
		for i in range(num_freqs):
			for j in range(total_chunks):
				if j > total_chunks - 1:
					avg_vec = self.fft_db[i][chunk_size * j:]
					mu = np.mean(avg_vec)
					avg_mat[i][j] = mu
				avg_vec = self.fft_db[i][chunk_size * j: chunk_size * (j+1)]
				mu = np.mean(avg_vec)
				avg_mat[i][j] = mu
		self.chunk_fft = avg_mat

	def set_rank_2d(self): 
		a = np.zeros(self.chunk_fft.shape)
		for row in range(self.chunk_fft.shape[1]):
			a[:, row] = np.abs(rankdata(self.chunk_fft[:, row], method='min') - (self.chunk_fft.shape[0])) + 1
		self.rank = a

	def set_sparsity(self):
		sparse_vec = np.zeros((1, self.rank.shape[1]))
		for i in range(self.rank.shape[1]):
			mu = np.mean(self.rank.T[i])
			if mu == self.freq_bins:
				sparse_vec[0, i] = 0
			else:
				sparse_vec[0, i] = 1
		self.sparse_vec = sparse_vec

In [44]:
s = Signal(path, 1024, 1024)

In [45]:
s.set_chunk(5)
s.set_rank_2d()
s.set_sparsity()

In [46]:
s.chunk_fft

array([[-50.48006058, -50.48006058, -50.48006058, ..., -15.74076748,
        -10.05226421, -20.81770325],
       [-50.48006058, -50.48006058, -50.48006058, ...,   6.88460493,
          4.32526016,   3.81143379],
       [-50.48006058, -50.48006058, -50.48006058, ...,  12.71502495,
         10.69186878,   9.91062355],
       ...,
       [-50.48006058, -50.48006058, -50.48006058, ..., -50.48006058,
        -50.48006058, -50.48006821],
       [-50.48006058, -50.48006058, -50.48006058, ..., -50.48006058,
        -50.48006058, -50.48006821],
       [-50.48006058, -50.48006058, -50.48006058, ..., -50.48006058,
        -50.48006058, -50.48006821]])

In [53]:
sr = librosa.get_samplerate(path)
sr1 = librosa.get_samplerate(path1)

In [26]:
sr, sr1

(44100, 44100)

In [55]:
y, sr = librosa.load(path, sr=sr)
y1, sr = librosa.load(path1, sr=sr)

In [7]:
onset_env = librosa.onset.onset_strength(y, sr=sr)
tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)

/anaconda3/envs/audio/lib/python3.5/site-packages/librosa/beat.py:309: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  hop_length=hop_length))


In [23]:
onset_env1 = librosa.onset.onset_strength(y1, sr=sr)
tempo1 = librosa.beat.tempo(onset_envelope=onset_env1, sr=sr)

/anaconda3/envs/audio/lib/python3.5/site-packages/librosa/beat.py:309: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  hop_length=hop_length))


In [24]:
tempo1

array([63.80208333])

In [22]:
tempo

array([120.18531977])

In [11]:
tempo * 2.483

array([298.42014898])

In [17]:
298.42 / 4

74.605

In [10]:
149.3 / 60

2.4883333333333337

In [12]:
1024 / 44100

0.023219954648526078

In [34]:
(1 / (1024 / 44100)) * 5

215.33203125

In [13]:
tempo / 60

array([2.00308866])

In [14]:
(tempo / 60) / (1024 / 44100)

array([86.26583011])

In [15]:
6432 / 86.265

74.56094592244827

In [56]:
D = np.abs(librosa.core.stft(y, n_fft=1024, hop_length=1024))
D_db = librosa.amplitude_to_db(D)
D1 = np.abs(librosa.core.stft(y1, n_fft=1024, hop_length=1024))
D1_db = librosa.amplitude_to_db(D1)

In [40]:
# D.shape
6432 / 216

29.77777777777778

In [63]:
# D_db.shape
frequency_range = np.array([i * (sr / D_db.shape[0]) for i in range(513)])

In [96]:
frequency_range

array([    0.        ,    85.96491228,   171.92982456,   257.89473684,
         343.85964912,   429.8245614 ,   515.78947368,   601.75438596,
         687.71929825,   773.68421053,   859.64912281,   945.61403509,
        1031.57894737,  1117.54385965,  1203.50877193,  1289.47368421,
        1375.43859649,  1461.40350877,  1547.36842105,  1633.33333333,
        1719.29824561,  1805.26315789,  1891.22807018,  1977.19298246,
        2063.15789474,  2149.12280702,  2235.0877193 ,  2321.05263158,
        2407.01754386,  2492.98245614,  2578.94736842,  2664.9122807 ,
        2750.87719298,  2836.84210526,  2922.80701754,  3008.77192982,
        3094.73684211,  3180.70175439,  3266.66666667,  3352.63157895,
        3438.59649123,  3524.56140351,  3610.52631579,  3696.49122807,
        3782.45614035,  3868.42105263,  3954.38596491,  4040.35087719,
        4126.31578947,  4212.28070175,  4298.24561404,  4384.21052632,
        4470.1754386 ,  4556.14035088,  4642.10526316,  4728.07017544,
      

In [76]:
# frequency_range[(frequency_range < 20000) & (frequency_range > 20)]
np.where((frequency_range < 20000) & (frequency_range > 20))

(array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
         27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
         40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
         53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
         66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
         79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
         92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
        105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
        118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
        131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
        144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
        157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
        170, 171, 172, 173, 174, 175, 176, 177, 178

In [58]:
avg_vector = np.zeros((1, 216))
for i in range(int(np.ceil(D_db.shape[1] / 216))):
    if i > ((D_db.shape[1] / 216) - 1):
        avg_vector = D_db[150][216*i:]
        mu = np.mean(avg_vector)
#         print(mu)
    avg_vector = D_db[150][216*i: 216*(i+1)]
    mu = np.mean(avg_vector)
#     print(mu)

In [115]:
def fft_chunk(y, seconds, window_size, sr):
    fft_length = y.shape[1]
    num_freqs = y.shape[0]
    chunk_size = int(np.ceil((1 / (window_size / sr)) * seconds))
    total_chunks = int(np.ceil(fft_length / chunk_size))
    avg_mat = np.zeros((num_freqs, total_chunks))
    avg_vec = np.zeros((1, chunk_size))
    for i in range(num_freqs):
        for j in range(total_chunks):
            if j > total_chunks - 1:
                avg_vec = y[i][chunk_size * j:]
                mu = np.mean(avg_vec)
                avg_mat[i][j] = mu
            avg_vec = y[i][chunk_size * j: chunk_size * (j+1)]
            mu = np.mean(avg_vec)
            avg_mat[i][j] = mu
    return avg_mat

In [116]:
y_chunk = fft_chunk(D_db, 5, 1024, 44100)

In [117]:
y1_chunk = fft_chunk(D1_db, 5, 1024, 44100)

In [82]:
y_chunk

array([[-50.48006058, -50.48006058, -50.48006058, ..., -15.74076748,
        -10.05226421, -20.81770325],
       [-50.48006058, -50.48006058, -50.48006058, ...,   6.88460493,
          4.32526016,   3.81143379],
       [-50.48006058, -50.48006058, -50.48006058, ...,  12.71502495,
         10.69186878,   9.91062355],
       ...,
       [-50.48006058, -50.48006058, -50.48006058, ..., -50.48006058,
        -50.48006058, -50.48006821],
       [-50.48006058, -50.48006058, -50.48006058, ..., -50.48006058,
        -50.48006058, -50.48006821],
       [-50.48006058, -50.48006058, -50.48006058, ..., -50.48006058,
        -50.48006058, -50.48006821]])

In [99]:
r_y = rank_signal_2d(y_chunk)

In [118]:
r_y1 = rank_signal_2d(y1_chunk)

In [104]:
def audio_sparsity(r_y, min_y):
    sparse_vec = np.zeros((1, r_y.shape[1]))
    for i in range(r_y.shape[1]):
        mu = np.mean(r_y.T[i])
        if mu == min_y:
            sparse_vec[0, i] = 0
        else:
            sparse_vec[0, i] = 1
    return sparse_vec

In [110]:
s_y = audio_sparsity(r_y, r_y.shape[0])

In [119]:
s_y1 = audio_sparsity(r_y1, r_y1.shape[0])

In [141]:
def overlap(a, b):
    overlap_vec = a * b
    num_overlaps = np.sum(overlap_vec)
#     or_vec = np.where(a == b, 1 , 0)
    overlap_ratio = num_overlaps / overlap_vec.shape[1]
    return overlap_vec, num_overlaps, overlap_ratio

In [139]:
overlap

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [142]:
o, num_o, o_r = overlap(s_y, s_y1)

In [151]:
o, num_o, o_r

(array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]),
 1.0,
 0.03333333333333333)

In [128]:
y_chunk[0]

array([-50.48006058, -50.48006058, -50.48006058, -50.48006058,
       -50.48006058, -50.48006058, -50.48006058, -50.48006058,
       -50.48006058, -50.48006058, -50.48006058, -50.48006058,
       -38.95417023,  -8.55482864, -18.1507206 ,  -8.02236843,
       -17.91048431, -49.34363174, -50.48006058, -50.48006058,
       -50.48006058, -50.48006058, -50.48006058, -50.48006058,
       -50.48006058, -27.80969238,  -9.95785522, -15.74076748,
       -10.05226421, -20.81770325])

In [149]:
# y_chunk.dot(o.T) / num_o
y_chunk[4]

array([-50.48006058, -50.48006058, -50.48006058, -50.48006058,
       -50.48006058, -50.48006058, -50.48006058, -50.48006058,
       -50.48006058, -50.48006058, -50.48006058, -50.48006058,
       -33.29862976,  12.79619312,  12.36432266,  13.07997608,
        11.99641132, -34.66110611, -50.48006058, -50.48006058,
       -50.48006058, -50.48006058, -50.48006058, -50.48006058,
       -50.48006058, -18.38640594,  13.04644775,  13.53992176,
        12.71369171,  10.64991379])

In [164]:
# y_chunk.dot(o.T) - y1_chunk.dot(o.T)
# y1_chunk.dot(o.T)
# y1_chunk[0]
# y1_chunk[0] * s_y1
# (y1_chunk[0] * s_y1) * o
ycf = np.zeros((y_chunk.shape[0], 1))
ycf1 = np.zeros((y1_chunk.shape[0], 1))

for i in range(y1_chunk.shape[0]):
    ycf[i] = np.sum((y_chunk[i] * s_y) * o) / num_o
    ycf1[i] = np.sum((y1_chunk[i] * s_y1) * o) / num_o
    
# ycf

In [199]:
# ycf1 - ycf
rank_y = rank_signal_1d(ycf)
rank_y1 = rank_signal_1d(ycf1)
r_a = np.expand_dims(np.where(rank_y > 10, 1, 0), axis=1)
r_b = np.expand_dims(np.where(rank_y1 <= 10, 1, 0), axis=1)
v = ((r_a*r_b)*(ycf - ycf1)).flatten()

In [200]:
v

array([-34.37774372, -25.71929169,  -0.        ,  -0.        ,
        -0.        , -20.29273796, -21.11304092, -18.87050629,
        -0.        ,  -0.        ,  -0.        ,  -0.        ,
        -0.        ,  -0.        ,  -0.        ,  -0.        ,
        -0.        ,  -0.        ,  -0.        ,  -0.        ,
        -0.        ,  -0.        ,  -0.        ,  -0.        ,
        -0.        ,  -0.        ,  -0.        ,  -0.        ,
        -0.        ,  -0.        ,  -0.        ,  -0.        ,
        -0.        ,  -0.        ,  -0.        ,  -0.        ,
        -0.        ,  -0.        ,  -0.        ,  -0.        ,
        -0.        ,  -0.        ,  -0.        ,  -0.        ,
        -0.        ,  -0.        ,  -0.        ,  -0.        ,
        -0.        ,  -0.        ,  -0.        ,  -0.        ,
        -0.        ,  -0.        ,  -0.        ,  -0.        ,
        -0.        ,  -0.        ,  -0.        ,  -0.        ,
        -0.        ,  -0.        ,  -0.        ,  -0.  

In [185]:
# ycf1 - ycf
# rank_signal_1d(ycf1).shape
# (ycf1 - ycf).shape
((ycf-ycf1)*(r_a*r_b)).shape

(513, 513)

In [75]:
rank_signal_1d(y_chunk[0])

array([30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 10,  2,  7,  1,  6,
       11, 18, 17, 16, 15, 14, 13, 12,  9,  3,  5,  4,  8])

In [64]:
D_db.shape

(513, 6432)

In [59]:
avg_vector = np.zeros((1, 48))
for i in range(134):
    avg_vector = D_db[150][48*i: 48*(i+1)]
    mu = np.mean(avg_vector)
#     print(mu)

In [57]:
avg_vector = np.zeros((1, 216))
for i in range(int(np.ceil(D_db.shape[1] / 216))):
    if i > ((D1_db.shape[1] / 216) - 1):
        avg_vector = D1_db[150][216*i:]
        mu = np.mean(avg_vector)
        print(mu)
    avg_vector = D1_db[150][216*i: 216*(i+1)]
    mu = np.mean(avg_vector)
    print(mu)

-30.652927
-30.652927
-30.652927
-30.652927
-30.499521
-30.524275
-30.396774
-30.520191
-30.602787
-30.652927
-30.652927
-30.652927
-30.652927
-30.652927
-30.652927
-30.652927
-30.652927
-30.3479
-30.599857
-30.484507
-30.304426
-30.628422
-30.652927
-30.652927
-30.652927
-30.652927
-30.652927
-30.652927
-30.652927
-30.652925
-30.652925


In [56]:
librosa.core.samples_to_time(y, sr=44100).min()

-6.125314e-06

In [37]:
6432 / 1784

3.6053811659192827

In [40]:
22745*513

11668185

In [41]:
513*6432

3299616

In [111]:
y.shape[0] / 44100

149.33331065759637

In [109]:
y.shape

(6585599,)

In [112]:
(1024 / 44100) * 6432

149.35074829931972